### Install keras and tensorflow:

In [1]:
!conda install -c conda-forge keras --yes
!conda install -c conda-forge tensorflow --yes

### Import libraries:

In [60]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
import os

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

['04354.jpg', '02725.jpg', '07885.jpg', '09670.jpg', '06543.jpg', '00132.jpg', '00654.jpg', '08208.jpg', '09116.jpg', '06225.jpg', '02043.jpg', '04432.jpg', '02057.jpg', '05738.jpg', '04426.jpg', '03349.jpg', '00640.jpg', '00898.jpg', '09102.jpg', '06231.jpg', '01238.jpg', '07891.jpg', '09664.jpg', '06557.jpg', '07649.jpg', '00126.jpg', '04340.jpg', '02731.jpg', '01210.jpg', '07661.jpg', '08552.jpg', '09894.jpg', '03407.jpg', '04368.jpg', '05076.jpg', '02719.jpg', '05710.jpg', '03361.jpg', '00668.jpg', '07107.jpg', '08234.jpg', '06219.jpg', '01576.jpg', '07113.jpg', '08220.jpg', '01562.jpg', '05704.jpg', '03375.jpg', '03413.jpg', '05062.jpg', '01204.jpg', '09658.jpg', '07675.jpg', '08546.jpg', '09880.jpg', '07846.jpg', '06580.jpg', '05089.jpg', '04397.jpg', '05937.jpg', '02080.jpg', '01589.jpg', '00697.jpg', '00683.jpg', '05923.jpg', '02094.jpg', '04383.jpg', '07852.jpg', '06594.jpg', '02902.jpg', '08591.jpg', '09857.jpg', '00873.jpg', '00867.jpg', '08585.jpg', '09843.jpg', '02916.jpg'

### Data import:

In [103]:
'''img_path = 'food/00008.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)'''

folder_path = 'food/'

img_width, img_height = 224, 224

# load all images into a list
file_list = sorted(os.listdir(folder_path)[0:10])
images = []
for img in file_list:
    if img == '.DS_Store':  # ignore stupid fookin mac file that wont go away
        continue
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    images.append(img)

# stack up images list to pass for prediction
images = np.vstack(images)

### Load pre-trained model:

In [82]:
model = ResNet50(weights='imagenet')

### Predict:

In [124]:
predictions = model.predict(images, batch_size=64)

### Save:

In [132]:
# Save top 3 label probabilities as the 3 features describing image:
predictions_decoded = np.array(decode_predictions(predictions, top=3))
predictions_decoded = predictions_decoded[:, :, 1]

df = pd.DataFrame(predictions_decoded, columns=['first', 'second', 'third'])
print(df)

df.to_csv('food_features', index=False)

           first       second         third
0     coral_reef     mushroom        bolete
1          dough  brain_coral          coil
2         goblet      thimble       pitcher
3     leafhopper       cicada       spindle
4           pick       shield     spotlight
5      soup_bowl          cup       pitcher
6  jigsaw_puzzle  loudspeaker  water_bottle
7          screw   sea_urchin   beer_bottle
8     hair_slide     fountain        bonnet
9       bolo_tie      peacock  water_bottle


In [133]:
df = pd.read_csv('food_features')
print(df)

           first       second         third
0     coral_reef     mushroom        bolete
1          dough  brain_coral          coil
2         goblet      thimble       pitcher
3     leafhopper       cicada       spindle
4           pick       shield     spotlight
5      soup_bowl          cup       pitcher
6  jigsaw_puzzle  loudspeaker  water_bottle
7          screw   sea_urchin   beer_bottle
8     hair_slide     fountain        bonnet
9       bolo_tie      peacock  water_bottle
